In [5]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [6]:
env = GridworldEnv()

In [63]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    prev_V = V.copy()
    cnt = 0
    while True:
        # Will first use the synchronous method and then try with the asynchronous method
        # for each action and for each state compute the vs 
        for s in range(env.nS):
            for a in range(len(policy[s])):
                prob, next_state, reward, _ = env.P[s][a][0]
                V[s] += (policy[s][a] * (reward + np.sum(prob * prev_V[next_state])))
        
        if np.allclose(prev_V, V, atol=theta, rtol=0):
            break
        prev_V = V.copy()
        V = np.zeros(env.nS)
    return np.array(V)

In [64]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)
print(v)

[  0.         -13.99989315 -19.99984167 -21.99982282 -13.99989315
 -17.99986052 -19.99984273 -19.99984167 -19.99984167 -19.99984273
 -17.99986052 -13.99989315 -21.99982282 -19.99984167 -13.99989315   0.        ]


In [65]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)